In [5]:
# Load packages
suppressPackageStartupMessages({
  library(Seurat)
  library(SeuratData)
  library(ggplot2)
  library(patchwork)
  library(scales)
  library(dplyr)
  library(reshape2)
})

In [6]:
# Load dataset
InstallData(ds = "thp1.eccite")
eccite <- LoadData(ds = "thp1.eccite")
eccite <- UpdateSeuratObject(eccite)

Warning message:
“The following packages are already installed and will not be reinstalled: thp1.eccite”
Validating object structure

Updating object slots

Ensuring keys are in the proper structure

Warning message:
“Assay RNA changing from Assay to Assay”
Warning message:
“Assay ADT changing from Assay to Assay”
Warning message:
“Assay HTO changing from Assay to Assay”
Warning message:
“Assay GDO changing from Assay to Assay”
Ensuring keys are in the proper structure

Ensuring feature names don't have underscores or pipes

Updating slots in RNA

Updating slots in ADT

Updating slots in HTO

Updating slots in GDO

Validating object structure for Assay ‘RNA’

Validating object structure for Assay ‘ADT’

Validating object structure for Assay ‘HTO’

Validating object structure for Assay ‘GDO’

Object representation is consistent with the most current Seurat version



In [7]:
# Preprocessing
# Protein
eccite <- Seurat::NormalizeData(
  object = eccite,
  assay = "ADT",
  normalization.method = "CLR",
  margin = 2)

# RNA
DefaultAssay(object = eccite) <- 'RNA'
eccite <- NormalizeData(object = eccite) %>% FindVariableFeatures() %>% ScaleData()

# Create a random matrix with one row per cell and 50 principal components
set.seed(123)
cells <- Cells(eccite)
rand_mat <- matrix(rnorm(length(cells) * 50, 0, 1), nrow = length(cells), ncol = 50)
rownames(rand_mat) <- cells
eccite[["pca"]] <- CreateDimReducObject(
  embeddings = rand_mat,
  key = "PC_",
  assay = DefaultAssay(eccite)
)


Normalizing across cells



Centering and scaling data matrix

Warning message:
“No columnames present in cell embeddings, setting to 'PC_1:50'”


In [8]:
# Mitigating confounding effects
eccite<- CalcPerturbSig(
  object = eccite,
  assay = "RNA",
  slot = "data",
  gd.class ="gene",
  nt.cell.class = "NT",
  reduction = "pca",
  ndims = 40,
  num.neighbors = 20,
  split.by = "replicate",
  new.assay.name = "PRTB")

Processing rep1

Processing rep3

Processing rep2

Warning message:
“Layer counts isn't present in the assay object; returning NULL”


In [ ]:
# Prepare PRTB assay for dimensionality reduction:
# Normalize data, find variable features and center data
DefaultAssay(object = eccite) <- 'PRTB'

# identify cells with no detectable perturbation
eccite <- RunMixscape(
  object = eccite,
  assay = "PRTB",
  slot = "scale.data",
  labels = "gene",
  nt.class.name = "NT",
  min.de.genes = 5,
  iter.num = 10,
  de.assay = "RNA",
  verbose = F,
  prtb.type = "KO")

For a (much!) faster implementation of the Wilcoxon Rank Sum Test,
(default method for FindMarkers) please install the presto package
--------------------------------------------
install.packages('devtools')
devtools::install_github('immunogenomics/presto')
--------------------------------------------
After installation of presto, Seurat will automatically use the more 
efficient implementation (no further action necessary).
This message will be shown once per session

Warning message:
“Different cells and/or features from existing assay PRTB”
Warning message:
“Layer counts isn't present in the assay object; returning NULL”


number of iterations= 198 
number of iterations= 361 
number of iterations= 258 
number of iterations= 30 
number of iterations= 15 
number of iterations= 24 
number of iterations= 16 
number of iterations= 16 
number of iterations= 124 
number of iterations= 109 
number of iterations= 105 
number of iterations= 104 
number of iterations= 103 
number of iterations= 103 
number of iterations= 27 
number of iterations= 17 
number of iterations= 17 
number of iterations= 27 
number of iterations= 25 
number of iterations= 25 
number of iterations= 25 
number of iterations= 99 
number of iterations= 93 
number of iterations= 93 
number of iterations= 49 
number of iterations= 36 
number of iterations= 36 
number of iterations= 31 
number of iterations= 20 
number of iterations= 20 
number of iterations= 18 
number of iterations= 16 
number of iterations= 58 
number of iterations= 30 
